In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()

Plots.GRBackend()

In [3]:
import LCPSim
import LearningMPC
import BoxValkyries
reload("LearningMPC")
reload("BoxValkyries")

In [5]:
robot = BoxValkyries.BoxValkyrie(true, LCPSim.planar_revolute_base);
mvis = MechanismVisualizer(robot)
open(mvis.visualizer)

Listening on 127.0.0.1:7004...
zmq_url=tcp://127.0.0.1:6004
web_url=http://127.0.0.1:7004/static/


MeshCat Visualizer at http://127.0.0.1:7004/static/ with path /meshcat/robot

In [6]:
net, lqrsol, mpc_param_data = jldopen("box-val-revolute.jld2") do file
    (file["net"], file["lqrsol"], file["mpc_params"])
end

(Chain(Dense(22, 64, NNlib.elu), Dense(64, 64, NNlib.elu), Dense(64, 11)), LearningMPC.LQRSolution, Dict{String,Any}(Pair{String,Any}("horizon", 10),Pair{String,Any}("Δt", 0.05)))

In [7]:
xstar = BoxValkyries.nominal_state(robot)

net_controller = x -> Flux.Tracker.data(net(state_vector(x)))

mpc_params = LearningMPC.MPCParams(
    Δt=mpc_param_data["Δt"],
    horizon=mpc_param_data["horizon"],
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=60, 
        MIPGap=1e-1, 
        MIPGapAbs=5e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Academic license - for non-commercial use only
Academic license - for non-commercial use only


LearningMPC.MPCParams{Gurobi.GurobiSolver,Gurobi.GurobiSolver}
  Δt: Float64 0.05
  horizon: Int64 10
  mip_solver: Gurobi.GurobiSolver
  lcp_solver: Gurobi.GurobiSolver


In [9]:
srand(6)
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.0, 3.0)
set_configuration!(mvis, configuration(x0))
sleep(3)
results = LCPSim.simulate(x0, 
    net_controller,
    robot.environment, mpc_params.Δt, 50, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.1)

In [8]:
LearningMPC.playback(mvis, results, 0.05)